In [423]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [424]:
N = input('Ingrese el número de archivo deseado para el snapshot 125 de la multidark galaxy:')

Ingrese el número de archivo deseado para el snapshot 125 de la multidark galaxy:105


In [425]:
Archivo = f"/home/seba/SAG/gal_125_SAG-7.128_BOX_{N}.hdf5"

In [426]:
with h5py.File(Archivo, "r") as archivo:
    for datos in archivo.keys():
        print(datos)

FeH_hot_halo
FeH_stars
ForestID
GalaxyHostID
GalaxyStaticID
Galaxy_Type
Halo
HaloID
InstantSfrBulge
InstantSfrQuies
MZ_ICstars
MZ_gas_bulge
MZ_gas_disk
MZ_hot_halo
MZ_stars_bulge
MZ_stars_disk
M_ICstars
M_gas_bulge
M_gas_disk
M_hot
M_star_bulge
M_star_disk
Magnitudes
MainHaloID
Mbh
OH_gas_disk_bulge
Rhalf_bulge
Rhalf_disk
SFR
SFR_bulge
SFR_bulge_DI
T_stars
Vx
Vy
Vz
X
Y
Z


In [427]:
with h5py.File(Archivo, "r") as archivo:
    MainID = archivo['MainHaloID'][:].ravel()
    HaloID = archivo['HaloID'][:].ravel()
    R200 = archivo['Halo/R200c'][:].ravel()*0.674
    G_type = archivo['Galaxy_Type'][:].ravel()
    r = archivo['Magnitudes/Mag_rS'][:].ravel()
    x = archivo['X'][:].ravel()*0.674
    y = archivo['Y'][:].ravel()*0.674
    z = archivo['Z'][:].ravel()*0.674
    Vx = archivo['Vx'][:].ravel()
    Vy = archivo['Vy'][:].ravel()
    Vz = archivo['Vz'][:].ravel()   
    Vmax = archivo['Halo/Vmax'][:].ravel()

In [428]:
# Crear un DataFrame de pandas con el conjunto de datos
data = pd.DataFrame({'MainHaloID':MainID, 'HaloID': HaloID, 'Galaxy_Type': G_type, 'r':r, 'X':x, 'Y':y, 'Z':z, 'Vx':Vx, 'Vy':Vy, 'Vz':Vz, 'R200c':R200})
data.to_csv(f'Simulation_{N}.csv')
# Filtrar por número de galaxias pertenecientes a cada halo
#groups_filter = data.groupby('MainHaloID').filter(lambda x: 3 < len(x) <= 10 and x['r'].max() <= -18 and (x['r'].max() - x['r'].min()) <= 3)
#groups_filter

In [429]:
# Filtrar por número de galaxias pertenecientes a cada halo
groups_filter = data.groupby('MainHaloID').filter(lambda x: 3 < len(x) <= 10 and (x['r'].max() - x['r'].min()) <= 3)
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c
9304,12569484796,12569484796,0,-18.558458,14081.728516,90029.039062,308458.093750,202.160004,-128.550003,512.530029,61.794724
9305,12569484796,12569484796,2,-16.605272,14069.418945,90039.187500,308453.968750,104.273979,-121.301773,588.281921,32.337631
9306,12569484796,12569484796,2,-16.690390,14091.757812,90063.578125,308517.750000,168.965225,-159.557907,524.801270,32.713341
9307,12569484796,12569484797,1,-15.826831,14063.948242,90062.875000,308485.812500,172.669998,-214.029999,502.410004,36.239029
53301,12573541390,12573541390,0,-21.044460,29068.468750,370663.531250,198927.203125,-364.170013,197.240005,536.090027,93.332703
...,...,...,...,...,...,...,...,...,...,...,...
1539939,12705148791,12705148790,1,-15.479196,653883.312500,563974.000000,245175.562500,-136.080002,321.910004,-142.820007,32.925137
1558100,12706599110,12706599110,0,-19.090412,633284.000000,667142.437500,639281.937500,-85.849998,52.430000,73.709999,75.029305
1558101,12706599110,12706599110,2,-18.603291,633279.687500,667208.125000,639311.000000,-108.571312,27.967636,144.829819,48.034649
1558102,12706599110,12706599111,1,-17.523037,633290.062500,667188.187500,639325.187500,-58.520000,-43.259998,89.809998,44.352993


In [430]:
groups_filter['Distancia XY'] = np.sqrt(groups_filter['X']**2 + groups_filter['Y']**2)
groups_filter['Distancia YZ'] = np.sqrt(groups_filter['Y']**2 + groups_filter['Z']**2)
groups_filter['Distancia XZ'] = np.sqrt(groups_filter['X']**2 + groups_filter['Z']**2)
groups_filter['m_rx'] = groups_filter['r'] + 5*np.log10(groups_filter['Distancia XY']*1000) - 5
groups_filter['m_ry'] = groups_filter['r'] + 5*np.log10(groups_filter['Distancia YZ']*1000) - 5
groups_filter['m_rz'] = groups_filter['r'] + 5*np.log10(groups_filter['Distancia XZ']*1000) - 5
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz
9304,12569484796,12569484796,0,-18.558458,14081.728516,90029.039062,308458.093750,202.160004,-128.550003,512.530029,61.794724,91123.664062,321327.90625,308779.343750,16.239698,18.976286,18.889780
9305,12569484796,12569484796,2,-16.605272,14069.418945,90039.187500,308453.968750,104.273979,-121.301773,588.281921,32.337631,91131.789062,321326.81250,308774.687500,18.193079,20.929461,20.842939
9306,12569484796,12569484796,2,-16.690390,14091.757812,90063.578125,308517.750000,168.965225,-159.557907,524.801270,32.713341,91159.343750,321394.84375,308839.406250,18.108618,20.844805,20.758272
9307,12569484796,12569484797,1,-15.826831,14063.948242,90062.875000,308485.812500,172.669998,-214.029999,502.410004,36.239029,91154.351562,321364.00000,308806.250000,18.972054,21.708153,21.621597
53301,12573541390,12573541390,0,-21.044460,29068.468750,370663.531250,198927.203125,-364.170013,197.240005,536.090027,93.332703,371801.593750,420670.28125,201039.828125,16.807098,17.075245,15.471951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539939,12705148791,12705148790,1,-15.479196,653883.312500,563974.000000,245175.562500,-136.080002,321.910004,-142.820007,32.925137,863498.750000,614961.56250,698336.937500,24.202118,23.465046,23.741127
1558100,12706599110,12706599110,0,-19.090412,633284.000000,667142.437500,639281.937500,-85.849998,52.430000,73.709999,75.029305,919852.000000,923991.56250,899850.000000,20.728180,20.737926,20.680439
1558101,12706599110,12706599110,2,-18.603291,633279.687500,667208.125000,639311.000000,-108.571312,27.967636,144.829819,48.034649,919896.687500,924059.06250,899867.625000,21.215405,21.225208,21.167603
1558102,12706599110,12706599111,1,-17.523037,633290.062500,667188.187500,639325.187500,-58.520000,-43.259998,89.809998,44.352993,919889.312500,924054.56250,899885.000000,22.295643,22.305450,22.247898


In [431]:
groups_filter['m_mean'] = (groups_filter['m_rx'] + groups_filter['m_ry'] + groups_filter['m_rz'])/3
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,m_mean
9304,12569484796,12569484796,0,-18.558458,14081.728516,90029.039062,308458.093750,202.160004,-128.550003,512.530029,61.794724,91123.664062,321327.90625,308779.343750,16.239698,18.976286,18.889780,18.035255
9305,12569484796,12569484796,2,-16.605272,14069.418945,90039.187500,308453.968750,104.273979,-121.301773,588.281921,32.337631,91131.789062,321326.81250,308774.687500,18.193079,20.929461,20.842939,19.988493
9306,12569484796,12569484796,2,-16.690390,14091.757812,90063.578125,308517.750000,168.965225,-159.557907,524.801270,32.713341,91159.343750,321394.84375,308839.406250,18.108618,20.844805,20.758272,19.903898
9307,12569484796,12569484797,1,-15.826831,14063.948242,90062.875000,308485.812500,172.669998,-214.029999,502.410004,36.239029,91154.351562,321364.00000,308806.250000,18.972054,21.708153,21.621597,20.767267
53301,12573541390,12573541390,0,-21.044460,29068.468750,370663.531250,198927.203125,-364.170013,197.240005,536.090027,93.332703,371801.593750,420670.28125,201039.828125,16.807098,17.075245,15.471951,16.451431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539939,12705148791,12705148790,1,-15.479196,653883.312500,563974.000000,245175.562500,-136.080002,321.910004,-142.820007,32.925137,863498.750000,614961.56250,698336.937500,24.202118,23.465046,23.741127,23.802765
1558100,12706599110,12706599110,0,-19.090412,633284.000000,667142.437500,639281.937500,-85.849998,52.430000,73.709999,75.029305,919852.000000,923991.56250,899850.000000,20.728180,20.737926,20.680439,20.715515
1558101,12706599110,12706599110,2,-18.603291,633279.687500,667208.125000,639311.000000,-108.571312,27.967636,144.829819,48.034649,919896.687500,924059.06250,899867.625000,21.215405,21.225208,21.167603,21.202738
1558102,12706599110,12706599111,1,-17.523037,633290.062500,667188.187500,639325.187500,-58.520000,-43.259998,89.809998,44.352993,919889.312500,924054.56250,899885.000000,22.295643,22.305450,22.247898,22.282997


In [432]:
# Filtrar por magnitud aparente
groups_filter = groups_filter.groupby('MainHaloID').filter(lambda x: (14 < x['m_rx']).all() and (x['m_rx'] <= 17.77).all())
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,m_mean
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,97.818001,150534.718750,343754.87500,370527.34375,15.521128,17.314188,17.477045,16.770788
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,38.781277,150509.140625,343741.15625,370499.71875,17.726284,19.519630,19.682407,18.976107
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,64.608269,150556.937500,343708.03125,370494.50000,16.319248,18.111694,18.274654,17.568533
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,46.892776,150537.296875,343761.03125,370533.87500,17.697754,19.490818,19.653675,18.947416


In [433]:
#Identificar la galaxia host y las galaxias satélite en cada grupo
def calcular_dispersion(group):
    galaxia_host = group[group['Galaxy_Type'] == 0]
    galaxias_satelite = group[group['Galaxy_Type'].isin([1, 2])]

    #Restar las velocidades de las galaxias satélite respecto a la galaxia host
    dispersion_velocidades = galaxias_satelite[['Vx', 'Vy', 'Vz']] - galaxia_host[['Vx', 'Vy', 'Vz']].values[0]

    #Calcular la magnitud de la dispersión de velocidades0
    dispersion_magnitud = dispersion_velocidades.apply(lambda x: (x**2).sum()**0.5, axis=1)

    return dispersion_magnitud.mean()

#Aplica la función a cada grupo y obtén la dispersión de velocidades promedio
dispersion_promedio = groups_filter.groupby('MainHaloID').apply(calcular_dispersion)

print(dispersion_promedio)


MainHaloID
12597437366    148.827863
dtype: float64


In [434]:
groups_filter['Dispersion'] = groups_filter['MainHaloID'].map(dispersion_promedio)
groups_filter

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,m_mean,Dispersion
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,97.818001,150534.718750,343754.87500,370527.34375,15.521128,17.314188,17.477045,16.770788,148.827863
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,38.781277,150509.140625,343741.15625,370499.71875,17.726284,19.519630,19.682407,18.976107,148.827863
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,64.608269,150556.937500,343708.03125,370494.50000,16.319248,18.111694,18.274654,17.568533,148.827863
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,46.892776,150537.296875,343761.03125,370533.87500,17.697754,19.490818,19.653675,18.947416,148.827863


In [435]:
cg = groups_filter.groupby('MainHaloID').filter(lambda x: x['Dispersion'].mean() < 1000)
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,R200c,Distancia XY,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,m_mean,Dispersion
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,97.818001,150534.718750,343754.87500,370527.34375,15.521128,17.314188,17.477045,16.770788,148.827863
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,38.781277,150509.140625,343741.15625,370499.71875,17.726284,19.519630,19.682407,18.976107,148.827863
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,64.608269,150556.937500,343708.03125,370494.50000,16.319248,18.111694,18.274654,17.568533,148.827863
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,46.892776,150537.296875,343761.03125,370533.87500,17.697754,19.490818,19.653675,18.947416,148.827863


In [436]:
#Para seleccionar solo el R200 de la galaxia host
cg['Host_R200'] = cg.groupby('MainHaloID')['R200c'].transform('first')
cg['Host_Distancia XY'] = cg.groupby('MainHaloID')['Distancia XY'].transform('first')
cg['Host_Distancia YZ'] = cg.groupby('MainHaloID')['Distancia YZ'].transform('first')
cg['Host_Distancia XZ'] = cg.groupby('MainHaloID')['Distancia XZ'].transform('first')

# Calcula el tamaño angular para cada dirección usando el R200 de la galaxia host en su grupo
cg['Tamaño Angular del grupo XY (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia XY']))*3600
cg['Tamaño Angular del grupo YZ (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia YZ']))*3600
cg['Tamaño Angular del grupo XZ (arcsec)'] = 2*np.arctan((2*cg['Host_R200'])/(cg['Host_Distancia XZ']))*3600

cg.drop('Host_R200', axis=1, inplace=True)
cg.drop('Host_Distancia XY', axis=1, inplace=True)
cg.drop('Host_Distancia YZ', axis=1, inplace=True)
cg.drop('Host_Distancia XZ', axis=1, inplace=True)
cg

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia YZ,Distancia XZ,m_rx,m_ry,m_rz,m_mean,Dispersion,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec)
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,...,343754.87500,370527.34375,15.521128,17.314188,17.477045,16.770788,148.827863,9.357167,4.097627,3.801552
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,...,343741.15625,370499.71875,17.726284,19.519630,19.682407,18.976107,148.827863,9.357167,4.097627,3.801552
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,...,343708.03125,370494.50000,16.319248,18.111694,18.274654,17.568533,148.827863,9.357167,4.097627,3.801552
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,...,343761.03125,370533.87500,17.697754,19.490818,19.653675,18.947416,148.827863,9.357167,4.097627,3.801552


In [437]:
def brillo(group):
    brillo_xy = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_rx']))) / (np.pi * group['Tamaño Angular del grupo XY (arcsec)'] ** 2))
    brillo_yz = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_ry']))) / (np.pi * group['Tamaño Angular del grupo YZ (arcsec)'] ** 2))
    brillo_xz = -2.5 * np.log10((np.sum(10 ** (-0.4 * group['m_rz']))) / (np.pi * group['Tamaño Angular del grupo XZ (arcsec)'] ** 2))

    # Calcular el brillo promedio y asignar el resultado a cada fila del grupo
    brillo_promedio = (brillo_xy + brillo_yz + brillo_xz) / 3
    group['Brillo Superficial'] = brillo_promedio

    return group

# Aplicar la función a cada grupo y luego combinar los resultados
cg = cg.groupby('MainHaloID').apply(brillo)
cg

/tmp/ipykernel_4922/3415725933.py:13: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cg = cg.groupby('MainHaloID').apply(brillo)


,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia XZ,m_rx,m_ry,m_rz,m_mean,Dispersion,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,...,370527.34375,15.521128,17.314188,17.477045,16.770788,148.827863,9.357167,4.097627,3.801552,21.014925
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,...,370499.71875,17.726284,19.519630,19.682407,18.976107,148.827863,9.357167,4.097627,3.801552,21.014925
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,...,370494.50000,16.319248,18.111694,18.274654,17.568533,148.827863,9.357167,4.097627,3.801552,21.014925
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,...,370533.87500,17.697754,19.490818,19.653675,18.947416,148.827863,9.357167,4.097627,3.801552,21.014925


In [438]:
cg.loc[cg['Brillo Superficial'] < 26.33]

,MainHaloID,HaloID,Galaxy_Type,r,X,Y,Z,Vx,Vy,Vz,...,Distancia XZ,m_rx,m_ry,m_rz,m_mean,Dispersion,Tamaño Angular del grupo XY (arcsec),Tamaño Angular del grupo YZ (arcsec),Tamaño Angular del grupo XZ (arcsec),Brillo Superficial
322929,12597437366,12597437366,0,-20.367056,144540.31250,42057.089844,341172.40625,277.549988,-151.229996,532.880005,...,370527.34375,15.521128,17.314188,17.477045,16.770788,148.827863,9.357167,4.097627,3.801552,21.014925
322930,12597437366,12597437366,2,-18.161530,144507.90625,42076.890625,341156.15625,272.719513,-186.237869,661.785034,...,370499.71875,17.726284,19.519630,19.682407,18.976107,148.827863,9.357167,4.097627,3.801552,21.014925
322931,12597437366,12597437365,1,-19.569256,144565.50000,42050.054688,341126.06250,220.779999,-42.060001,491.760010,...,370494.50000,16.319248,18.111694,18.274654,17.568533,148.827863,9.357167,4.097627,3.801552,21.014925
322932,12597437366,12597437364,1,-18.190464,144542.71875,42058.039062,341178.50000,225.550003,-305.220001,448.600006,...,370533.87500,17.697754,19.490818,19.653675,18.947416,148.827863,9.357167,4.097627,3.801552,21.014925


In [439]:
candidatos_fase3 = cg.loc[cg['Galaxy_Type'] == 0]
candidatos_fase3.to_csv(f'candidatos_fase3_{N}.csv')

In [440]:
candidates = pd.read_csv(f'candidatos_fase3_{N}.csv')
candidates
data = pd.read_csv(f'Simulation_{N}.csv')

In [441]:
try:
    compact_groups = pd.read_csv('compact_groups.csv')
except FileNotFoundError:
    # Si el archivo no existe, crea un DataFrame vacío
    compact_groups = pd.DataFrame()

for i in range(len(candidates)):
    data_reference_frame_changed = data.copy()
    data_reference_frame_changed['X'] -= candidates['X'][i]
    data_reference_frame_changed['Y'] -= candidates['Y'][i]
    data_reference_frame_changed['Z'] -= candidates['Z'][i]
    #Todas las galaxias que están dentro de la caja de lado 12R200
    caja = data_reference_frame_changed.loc[(abs(data_reference_frame_changed['X']) < 6* candidates['R200c'][i]) & (abs(data_reference_frame_changed['Y']) < 6* candidates['R200c'][i]) & (abs(data_reference_frame_changed['Z']) < 6* candidates['R200c'][i])]
    
    aislamiento = caja.loc[caja['MainHaloID'] != candidates['MainHaloID'][i]]
    print(len((aislamiento)))
    
    # Modificar el DataFrame original según la condición
    if len(aislamiento) != 0:
        candidates = candidates.drop(index=i)
    #Agregar la fila al DataFrame compact_groups
    compact_groups = compact_groups.append(candidates.loc[i:i], ignore_index=True)
# Guardar el DataFrame compact_groups en un archivo CSV
compact_groups.to_csv('compact_groups.csv', index=False)

0


/tmp/ipykernel_4922/1572733632.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compact_groups = compact_groups.append(candidates.loc[i:i], ignore_index=True)
